**CSE455 HW3 Ertugrul Kasikci 200104004097**

In [12]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
import numpy as np
  

df = datasets.load_breast_cancer()
X = df.data
y = df.target

print(type(df))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler to the training data
scaler.fit(X_train)

# Transform the training data
X_train = scaler.transform(X_train)

# Transform the test data
X_test = scaler.transform(X_test)


<class 'sklearn.utils._bunch.Bunch'>


**Part I: Select a dataset:**

I used datasets.load_breast_cancer() statement to get the breast cancer dataset, then divided it into X_train, X_test, y_train, y_test using the train_test_split function.

To get higher accuracy results, I used StandartScaler class so that all the attributes of the data will have similar effects on the result.

In [3]:
# MLP with one hidden layer
mlp1 = MLPClassifier(hidden_layer_sizes=(7,), early_stopping=True, random_state=42)
scores = cross_val_score(mlp1, X_train, y_train, cv=5)
print("Average cross-validation score of MLP with 1 hidden layer: ", scores.mean())

# MLP with two hidden layers
mlp2 = MLPClassifier(hidden_layer_sizes=(5, 10), early_stopping=True, random_state=42)
scores = cross_val_score(mlp2, X_train, y_train, cv=5)
print("Average cross-validation score of MLP with 2 hidden layers: ", scores.mean())

# MLP with three hidden layers
mlp3 = MLPClassifier(hidden_layer_sizes=(4, 6, 4), early_stopping=True, random_state=42)
scores = cross_val_score(mlp3, X_train, y_train, cv=5)
print("Average cross-validation score of MLP with 3 hidden layers: ", scores.mean())

# MLP with four hidden layers
mlp4 = MLPClassifier(hidden_layer_sizes=(5, 16, 9, 11), early_stopping=True,random_state=42)
scores = cross_val_score(mlp4, X_train, y_train, cv=5)
print("Average cross-validation score of MLP with 4 hidden layers: ", scores.mean())

# MLP with five hidden layers
mlp5 = MLPClassifier(hidden_layer_sizes=(6, 6, 10, 10, 13), early_stopping=True,random_state=42)
scores = cross_val_score(mlp5, X_train, y_train, cv=5)
print("Average cross-validation score of MLP with 5 hidden layers: ", scores.mean())



Average cross-validation score of MLP with 1 hidden layer:  0.8813186813186812
Average cross-validation score of MLP with 2 hidden layers:  0.8791208791208792
Average cross-validation score of MLP with 3 hidden layers:  0.6285714285714286
Average cross-validation score of MLP with 4 hidden layers:  0.6285714285714286
Average cross-validation score of MLP with 5 hidden layers:  0.7736263736263737


**Part II: Train a multi-layer perceptron:**

I trained 5 different MLPs with different numbers of layers and perceptrons. I used the early_stopping attribute to prevent overfitting.

To get their accuracy, I used cross-validation with cv = 5.

In [15]:
class customMLPClassifer(MLPClassifier):
    def resample_with_replacement(self, X_train, y_train, sample_weight):

        # normalize sample_weights if not already
        sample_weight = sample_weight / sample_weight.sum(dtype=np.float64)

        X_train_resampled = np.zeros((len(X_train), len(X_train[0])), dtype=np.float32)
        y_train_resampled = np.zeros((len(y_train)), dtype=int)
        for i in range(len(X_train)):
            # draw a number from 0 to len(X_train)-1
            draw = np.random.choice(np.arange(len(X_train)), p=sample_weight)

            # place the X and y at the drawn number into the resampled X and y
            X_train_resampled[i] = X_train[draw]
            y_train_resampled[i] = y_train[draw]

        return X_train_resampled, y_train_resampled

    def fit(self, X, y, sample_weight=None):
        if sample_weight is not None:
            X, y = self.resample_with_replacement(X, y, sample_weight)

        return self._fit(X, y, incremental=(self.warm_start and hasattr(self, "classes_")))

# This is a custom implementation of MLP which makes us able to use it as a estimator in AdaBoostClassifier.
mlp1 = customMLPClassifer(hidden_layer_sizes=(7,), early_stopping=True, random_state=42)
scores = cross_val_score(mlp1, X_train, y_train, cv=5)
print("Average cross-validation score of custom MLP with 1 hidden layer: ", scores.mean())


# Initialize the AdaBoost classifier
ada_clf = AdaBoostClassifier(estimator=mlp1, n_estimators=50, random_state=42)

# Fit the AdaBoost classifier to the training data
ada_clf.fit(X_train, y_train)

# Predict the test data
y_pred = ada_clf.predict(X_test)

# Compute the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of AdaBoost classifier with one-hidden-layer perceptron as the base classifier: ", accuracy)

Average cross-validation score of custom MLP with 1 hidden layer:  0.8813186813186812
Accuracy of AdaBoost classifier with one-hidden-layer perceptron as the base classifier:  0.956140350877193


**Part III: Train a multi-layer perceptron:**

To be able to use one hidden layer MLP as the base classifier of the AdaBoost classifier, I needed to implement it myself because the sample_weight attribute which is used when creating the AdaBoost classifier normally uses a decision tree. When I try to pass my one hidden layer MLP it gave me an error:

ValueError: NoneType doesn't support sample_weight.

customMLPClassifer solves this issue.

**customMLPClassifer:**

This custom MLP classifier has been overridden with a new fit method that takes sample_weight as an argument. If sample_weight is provided, the method will resample the training data according to these weights before fitting the model. This allows the MLP classifier to function as a weak learner in the AdaBoost algorithm

The accuracy of the AdaBoost learner us quite satisfying.

In [ ]:

''' # Create a Perceptron object
perceptron = Perceptron()

# Create a Random Forest Classifier object with the Perceptron as the base estimator
rf_clf = RandomForestClassifier(n_estimators=100, base_estimator=perceptron)

# Fit the Random Forest Classifier to the training data
rf_clf.fit(X_train, y_train)

# Predict the test data
y_pred = rf_clf.predict(X_test)

# Compute the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of Random Forest Classifier with Perceptron base estimator: ", accuracy) '''

**Part IV: Train a random decision forest where each decision in the forest is in turn is a trainable perceptron:**

As far as I understand to be able to implement this I need to make my own random forest and use perceptron in the decision trees instead of decisions. Because there is no way to change the base_estimator of random forest. Decision trees are deeply embedeed in them. I think this is quite hard thing to do and there is no enough time to do it.